1st attempt: [530, 163, 110, 146, 594, 109, 138, 115, 157, 533, 94, 145, 155, 164, 200, 129, 179, 201, 241, 522]

2nd attempt: [276, 394, 301, 595, 404, 311, 355, 404, 437, 394, 524, 357, 393, 374, 501, 237, 509, 356]

In [109]:
import numpy as np
from skimage import io
from SegmentationStatisticsCollector import convert_cell_labels_to_meshes, process_labels
import trimesh as tm
from skimage import io
from napari_process_points_and_surfaces import label_to_surface
import napari
import os
from tqdm import tqdm

from convert_to_cell_mesh import get_polydata_from_file, get_cell_ids, separate_cell_points_and_faces, write_unstructured_grid

%reload_ext autoreload
%autoreload 2

### Preprocessing of labels:

In [111]:
path_to_dir = "/nas/groups/iber/Users/Federico_Carrara/create_meshes/data/curated_labels_clean/"
image_name = "lung_new_sample_b_curated_segmentation_central_crop_relabeled_filled.tif"
img_path = os.path.join(path_to_dir, image_name)

save_dir = "../3d_tissues_preprocessing_and_segmentation/meshes/lung_new_sample_b/meshes_v2/preprocessed_cells"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

img = io.imread(img_path)
img = np.einsum('kij->ijk', img)
image = process_labels(img, 
                       dilation_iterations=2, 
                       erosion_iterations=4, 
                       output_directory=save_dir, 
                       overwrite=False, 
                       renumber=False)

Extending labels: 100%|██████████| 134/134 [10:54<00:00,  4.88s/it]


### Create Meshes

In [92]:
def create_combined_mesh(
        cell_labels, 
        labels_path, 
        make_meshes=True, 
        make_combined=True, 
        smoothing_iters=10,
        voxel_size=(0.21, 0.21, 0.39)):
    """
    Function to convert a preprocessed array of cell labels into meshes.

    args:
        cell_labels: an iterable containing the ids of the cells to be meshed, or "all" if all the labels 
        in the array need to be converted.
        labels_path: path to the array.
        make_meshes: a boolean telling if the meshes for the single cells are required.
        make_combined: a boolean telling if the combined mesh of cell_labels is required.
        smoothing_iters: an int representing the number of smoothing iterations to be done when creating the mesh.
        voxel_size: an iterable of len (3,) containing the voxel size in microns in (x, y, z) ordering.

    returns:
        meshes: a list whose elements are the meshes associated to each label.
        combined_mesh: the mesh associated to the combined labels.
        cell_labels: a list containing the indexes of the processed labels. 
    """
    
    # Read the image
    print("Loading labels...")
    labels = np.load(labels_path)

    # Set list of label ids
    print("Setting up things for conversion...")
    if cell_labels == "all":
        cell_labels = np.unique(labels)

    # Make a combined mask for all the labels in cell_labels
    combined_mask = np.isin(labels, cell_labels).astype(int)
    filtered_labels = labels * combined_mask

    # Transform voxel size in meters and convert to np.array
    voxel_size = [x*1e-6 for x in voxel_size]
    voxel_size = np.array(voxel_size)
    
    # Create meshese from the filtered_labels array using trimesh
    meshes = None
    if make_meshes:
        print("Converting single cells into meshes...")
        meshes = convert_cell_labels_to_meshes(filtered_labels, 
                                               voxel_resolution=voxel_size, 
                                               smoothing_iterations=smoothing_iters)
    
    # Get the combined mesh from combined_mask array
    combined_mesh = None
    if make_combined:
        print("Converting combined cells into meshes...")
        combined_mesh = convert_cell_labels_to_meshes(combined_mask, 
                                                      voxel_resolution=voxel_size, 
                                                      smoothing_iterations=smoothing_iters)
    
    return meshes, combined_mesh, cell_labels

In [102]:
# cells = [530, 163, 110, 146, 594, 109, 138, 115, 157, 533, 94, 145, 155, 164, 200, 129, 178, 201, 241, 522]
image_path = '../meshes_for_simulation/lung_new_sample_b/preprocessed_cells/processed_labels_d2_e4.npy'

meshes, combined_mesh, labels_list = create_combined_mesh(
    cell_labels="all", 
    labels_path=image_path,
    make_meshes=True, 
    make_combined=False, 
    smoothing_iters=100,
    voxel_size=(0.1625, 0.1625, 0.25))

Loading labels...
Setting up things for conversion...
Converting single cells into meshes...


Converting labels to meshes: 100%|██████████| 134/134 [12:27<00:00,  5.58s/it]


In [112]:
cells_ids = [2, 7, 22, 24, 29, 31, 35, 40, 45, 46, 48, 51, 65, 80, 120, 121, 122, 124]
meshes, combined_mesh, labels_list = create_combined_mesh(
    cell_labels=cells_ids, 
    labels_path=image_path,
    make_meshes=True, 
    make_combined=True, 
    smoothing_iters=100,
    voxel_size=(0.1625, 0.1625, 0.25))

Loading labels...
Setting up things for conversion...
Converting single cells into meshes...


Converting labels to meshes: 100%|██████████| 19/19 [01:29<00:00,  4.69s/it]


Converting combined cells into meshes...


Converting labels to meshes: 100%|██████████| 2/2 [00:48<00:00, 24.01s/it]


### Convert in .vtk format

In [83]:
# First export the meshes obtained so far into .stl format
def export_mesh(mesh, id, save_dir, file_type, overwrite=False) -> None:
    """
    Function to export meshes into .stl, .ply or .obj format

    args:
        mesh: the trimesh object to export
        label: either an int corresponding to the cell label, or "combined" if the combined mesh needs 
        to be saved
        save_dir: path to the location in which the mesh object will be exported. Single meshes will
        be saved in 'save_dir/single_cells', while combined ones in 'save_dir/combined'.
        file_type: the format of the file to be saved, chosen from [.ply, .stl, .obj]
    """
    # check that the file_type is among the allowed ones
    if file_type not in [".stl", ".ply", ".obj"]:
        raise ValueError("file_type is not correct. It must be among [.stl, .ply, .obj].")


    if id == "combined":
        # set the saving dir
        combined_save_dir = os.path.join(save_dir, "combined")
        if not os.path.exists(combined_save_dir):
            os.makedirs(combined_save_dir)       
        # save the mesh
        file_name = 'combined_cell' + file_type
        # check if the file already exists and if the user wants to overwrite
        if os.path.exists(os.path.join(combined_save_dir, file_name)) and not overwrite:
            print('Aborting export: The file already exists, and overwrite is set to False.')
            return
        else:
            mesh.export(os.path.join(combined_save_dir, file_name))
    elif isinstance(id, (int, np.int32, np.int64)):
        # set the saving dir
        single_cells_save_dir = os.path.join(save_dir, "single_cells")
        if not os.path.exists(single_cells_save_dir):
            os.makedirs(single_cells_save_dir)
        # save the mesh
        file_name = 'cell_{}'.format(id) + file_type
        if os.path.exists(os.path.join(single_cells_save_dir, file_name)) and not overwrite:
            print('Aborting export: the file already exists, and overwrite is set to False.')
            return
        else:
            mesh.export(os.path.join(single_cells_save_dir, file_name))
    else:
        raise ValueError("id must be either an int, np.int32, np.int64, or the string ""combined"".")

In [55]:
# export_mesh(
#     mesh=combined_mesh,
#     id="combined",
#     save_dir='./try',
#     file_type='.obj',
#     overwrite=True
# )

In [74]:
# labels_list=np.arange(len(meshes))

In [114]:
for mesh, id in tqdm(zip(meshes, labels_list), 
                     desc="Exporting meshes: ", 
                     total=len(labels_list)):
    export_mesh(
        mesh=mesh,
        id=id,
        save_dir='./try/stl_files',
        file_type='.stl',
        overwrite=True
    )
combined_mesh = combined_mesh[0]
export_mesh(
    mesh=combined_mesh,
    id="combined",
    save_dir='./try/stl_files',
    file_type='.stl',
    overwrite=True
)

Exporting meshes: 100%|██████████| 18/18 [00:00<00:00, 21.28it/s]


In [127]:
def convert_to_vtk(
    file_path, 
    output_dir
):
    """
    Function to convert a mesh object from .stl, .ply or .obj to .vtk

    args
        file_path: the path to the mesh file to convert
        output_dir: the path to the directory to save the converted mesh
    """
    #Read the mesh
    polydata = get_polydata_from_file(file_path)
    # print(polydata)

    #Set the output file name
    file_name = os.path.basename(file_path)
    file_extension = file_path[-4:]
    output_file_name = file_name.replace(file_extension, ".vtk")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_path = os.path.join(output_dir, output_file_name)

    #Get the cell to which each face belongs
    face_cell_id, point_cell_id = get_cell_ids(polydata)
    # print(face_cell_id, point_cell_id)

    #Separate the faces and points of each cell
    face_dic, point_dic = separate_cell_points_and_faces(polydata, face_cell_id, point_cell_id)
    # print(face_dic, point_dic)

    #Get the cells unstructured grids
    write_unstructured_grid(polydata, face_dic, point_dic, output_path)

In [128]:
out_dir = "./try/vtk_files/"
# convert_to_vtk("try/combined/combined_cell.stl", os.path.join(out_dir, "combined"))

file_paths = os.listdir('./try/stl_files/single_cells')
for fp in tqdm(file_paths):
    convert_to_vtk(os.path.join('./try/stl_files/single_cells', fp),
                   os.path.join(out_dir, "single_cells"))

100%|██████████| 18/18 [00:05<00:00,  3.29it/s]


### Visualise

In [ ]:
vertices, faces = combined_mesh[0].vertices, combined_mesh[0].faces
viewer = napari.Viewer()
viewer.add_surface((vertices, faces), name=f'large_mesh')

In [ ]:
for i, k in enumerate(cells):
    vertices, faces = labels_list[i][0].vertices, labels_list[i][0].faces
    viewer.add_surface((vertices, faces), name=f'mesh {k}')

### Export

In [ ]:
for i, k in enumerate(cells):
    labels_list[i][0].export(f'/Users/antanas/BC_Project/Simulation_Experiment_3/cell_{k-1}.stl')
combined_mesh[0].export(f'/Users/antanas/BC_Project/Simulation_Experiment_3/large_mesh.stl')

### Reading in to visualise

In [ ]:
path = '/Users/antanas/BC_Project/Simulation_Experiment_2'
viewer_1 = napari.Viewer()

for file in os.listdir('/Users/antanas/BC_Project/Simulation_Experiment_2'):
    mesh = tm.load(os.path.join('/Users/antanas/BC_Project/Simulation_Experiment_2', file))
    vertices, faces = mesh.vertices, mesh.faces
    viewer_1.add_surface((vertices, faces), name=os.path.basename(file))

In [ ]:
import numpy as np
import trimesh as tm
from skimage import io
from napari_process_points_and_surfaces import label_to_surface

path = '/Users/antanas/BC_Project/Control_Segmentation/276_mesh_sim.tif'
labels = io.imread(path)
mask = labels * (labels == 276)


mesh_lst = convert_cell_labels_to_meshes(mask, voxel_resolution=np.array([0.21, 0.21, 0.39]), smoothing_iterations=10)

# # Create a boolean array where True corresponds to the label 530
# mask = labels * (labels == 530) == 530

# # Create a mesh from the mask using trimesh
# mesh = label_to_surface(mask)

# print(mesh)



In [ ]:
mesh_lst[0].export('276_mesh.stl')

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()
viewer

In [ ]:
cells = [530, 163, 110, 146, 594, 109, 138, 115, 157, 533, 94, 145, 155, 164, 200, 129, 179, 201, 241, 522]

In [ ]:
import trimesh as tm
mesh_list = []

for i in cells:
    mesh_list.append(tm.load_mesh(f'/Users/antanas/BC_Project/Control_Segmentation_final/BC_control_s_5_e_2_d_5/cell_meshes/cell_{i-1}.stl'))

In [ ]:
viewer = napari.Viewer()
for i, k in enumerate([276, 394, 301, 595, 404, 311, 355, 404, 437, 394, 524, 357, 393, 374, 501, 237, 509, 356]):
    vertices, faces = mesh_list[i].vertices, mesh_list[i].faces
    viewer.add_surface((vertices, faces), name=f'mesh {k}')

In [ ]:
m178 = tm.load_mesh(f'/Users/antanas/BC_Project/Control_Segmentation_final/BC_control_s_5_e_2_d_5/cell_meshes/cell_{177}.stl')
vertices, faces = m178.vertices, m178.faces
viewer.add_surface((vertices, faces), name='mesh 178')

In [ ]:
vertices, faces = mesh_lst[0].vertices, mesh_lst[0].faces
viewer.add_surface((vertices, faces), name='Large Mesh')

In [ ]:
for i, k in enumerate(cells):
    mesh_list[i].export(f'/Users/antanas/BC_Project/Simulation_Experiment/cell_{k-1}.stl')
mesh_lst[0].export(f'/Users/antanas/BC_Project/Simulation_Experiment/large_mesh.stl')

In [ ]:
import os
import napari 

path = '/Users/antanas/BC_Project/Simulation_Experiment_2'
viewer_1 = napari.Viewer()

for file in os.listdir('/Users/antanas/BC_Project/Simulation_Experiment_2'):
    mesh = tm.load(os.path.join('/Users/antanas/BC_Project/Simulation_Experiment_2', file))
    vertices, faces = mesh.vertices, mesh.faces
    viewer_1.add_surface((vertices, faces), name=os.path.basename(file))